In [1]:
from nutri_checker.data.loader import load_nutri_data, FINAL_COLUMNS, COLUMNS_QUANTI, COLUMNS_QUALI
from nutri_checker.search.bm25 import NutriIndex, BM25Index
import pandas as pd
from dataclasses import dataclass
from loguru import logger

In [2]:
NUTRIDATA_PATH = "/Users/francois.weber/code/nutri-checker/data/table_Ciqual_2020_FR_20200707.xls"

In [3]:
df = load_nutri_data(NUTRIDATA_PATH)
nutri_index = NutriIndex(df)

In [7]:
from dataclasses import dataclass, asdict

@dataclass
class Aliment:
    grp_name_fr: str
    ssgrp_name_fr: str
    ssssgrp_name_fr: str
    name_fr: str
    grp_code: int
    ssgrp_code: int
    ssssgrp_code: int
    code: int
    # spec
    energy_kcal: float
    water_g: float
    prot_g: float
    glucid_g: float
    lipid_g: float
    sugar_g: float
    sugar_fructose_g: float
    sugar_galactose_g: float
    sugar_glucose_g: float
    sugar_lactose_g: float
    sugar_maltose_g: float
    sugar_saccharose_g: float
    amidon_g: float
    fiber_g: float
    alcool_g: float
    acid_sat_g: float
    acid_monoinsat_g: float
    acid_polyinsat_g: float
    cholesterol_g: float
    salt_g: float
    calcium_g: float
    iron_mg: float
    mg_mg: float
    vit_a_ug: float
    vit_d_ug: float
    vit_e_ug: float
    vit_k1_ug: float
    vit_k2_ug: float
    vit_c_mg: float
    vit_b1_mg: float
    vit_b2_mg: float
    vit_b3_mg: float
    vit_b5_mg: float
    vit_b6_mg: float
    vit_b9_ug: float
    vit_b12_ug: float
    # technical
    raw_food: bool
    
    def __post_init__(self):
        if not self.raw_food:
            logger.warning(f"Aliment {self.name_fr} is not raw food")
    
    @classmethod
    def from_row(cls, row):
        return cls(**row.to_dict())
    
    
    @classmethod
    def from_name_in_nutridata(cls, name: str, nutri_index: NutriIndex):
        if not any(cook_mode in name.lower() for cook_mode in ["cuit", "cuite", "cuisiné", "cuisinée"]):
            name = name + " " + "cru"
            
        aliment = nutri_index.retrieve(name, k=1).iloc[0].to_dict()
        return cls(**aliment)
    
    def get_nutritive_spec(self) -> pd.Series:
        return pd.Series(asdict(self))
        

In [8]:
@dataclass
class Ingredient:
    aliment: Aliment
    quantity_g: float
    
    def get_nutritive_spec(self) -> pd.Series:
        nutritive_unit_row = self.aliment.get_nutritive_spec()
        return pd.concat([nutritive_unit_row[COLUMNS_QUALI], nutritive_unit_row[COLUMNS_QUANTI] * 3])
    
    
@dataclass
class Pan:
    name: str
    weight: float
    

In [9]:
class IndustrialDish(Aliment):
    def __post_init__(self):
        pass
    
    def set_nutritive_spec(self, nutritive_details_from_package: dict) -> None:
        for key, value in nutritive_details_from_package.items():
            if not hasattr(self, key):
                logger.warning(f"Nutritive detail {key} not found in Aliment")
                continue
            setattr(self, key, value)
        

In [10]:
from typing import List

class Dish:
    def __init__(self, name, ingredients: List[Ingredient], total_weight_g=None, final_weight_g=None):
        self.name = name
        self.ingredients = ingredients
        self.total_weight_g = total_weight_g or sum(ing.quantity_g for ing in ingredients)
        # final weight after water evaporation. Default to total weight
        self.final_weight_g = final_weight_g if final_weight_g else self.total_weight_g

    def __str__(self):
        return f"{self.name} ({len(self.ingredients)} ingredients)"
    
    def __repr__(self):
        return f"{self.name} ({len(self.ingredients)} ingredients)"
    
    def __len__(self):
        return len(self.ingredients)

    def __iter__(self):
        return iter(self.ingredients)
    
    def __getitem__(self, idx):
        return self.ingredients[idx]
    
    def __setitem__(self, idx, value):
        self.ingredients[idx] = value
    
    def __delitem__(self, idx):
        del self.ingredients[idx]
    
    def __contains__(self, item):
        return item in self.ingredients
    
    def __add__(self, other):
        return Dish(self.name + " & " + other.name, self.ingredients + other.ingredients)
    
    def __iadd__(self, other):
        self.ingredients += other.ingredients
        return self
    
    def get_portion(self, portion_g):
        return Dish(self.name, [Ingredient(ing.aliment, portion_g * ing.quantity_g / self.final_weight_g) for ing in self.ingredients], total_weight_g=portion_g)
    
    def get_nutri(self):
        return pd.DataFrame(data=[ing.get_nutritive_spec() for ing in self.ingredients])

In [14]:
d = Dish("test", ingredients=[Ingredient(Aliment.from_name_in_nutridata("Pomme", nutri_index), 300), Ingredient(Aliment.from_name_in_nutridata("Banane", nutri_index), 100)])

In [16]:
d.get_nutri()

,grp_name_fr,ssgrp_name_fr,ssssgrp_name_fr,name_fr,grp_code,ssgrp_code,ssssgrp_code,code,energy_kcal,water_g,...,vit_k1_ug,vit_k2_ug,vit_c_mg,vit_b1_mg,vit_b2_mg,vit_b3_mg,vit_b5_mg,vit_b6_mg,vit_b9_ug,vit_b12_ug
0,"fruits, légumes, légumineuses et oléagineux",fruits,fruits crus,"pomme, pulpe, crue",2,204,20401,13050,113.625,260.1,...,1.8,0.0,12.00,0.057,0.084,0.273,0.213,0.111,0.0,0.0
1,"fruits, légumes, légumineuses et oléagineux",fruits,fruits crus,"banane, pulpe, crue",2,204,20401,13005,271.500,227.4,...,0.0,0.0,21.48,0.162,0.000,1.170,0.930,0.540,57.0,0.0


In [ ]:
class Menu:
    def __init__(self, name, dishes: List[Dish]):
        self.name = name
        self.dishes = dishes
    
    def __str__(self):
        return f"{self.name} ({len(self.dishes)} dishes)"
    
    def __repr__(self):
        return f"{self.name} ({len(self.dishes)} dishes)"
    
    def __len__(self):
        return len(self.dishes)

    def __iter__(self):
        return iter(self.dishes)
    
    def __getitem__(self, idx):
        return self.dishes[idx]
    
    def __setitem__(self, idx, value):
        self.dishes[idx] = value
    
    def __delitem__(self, idx):
        del self.dishes[idx]
    
    def __contains__(self, item):
        return item in self.dishes
    
    def __add__(self, other):
        return Menu(self.name + " & " + other.name, self.dishes + other.dishes)
    
    def __iadd__(self, other):
        self.dishes += other.dishes
        return self
    
    def get_portion(self, portion_g):
        return Menu(self.name, [dish.get_portion(portion_g) for dish in self.dishes])
    
    def get_nutri(self):
        return pd.DataFrame(data=[dish.get_nutri() for dish in self.dishes]).sum()

In [14]:
a = Dish("a", [
    Ingredient(Aliment.from_row(df.iloc[0]), 100),
    Ingredient(Aliment.from_row(df.iloc[10]), 100)
    ])

In [15]:
a.get_nutri()

grp_name_fr           fruits, légumes, légumineuses et oléagineuxfru...
ssgrp_name_fr                                            légumeslégumes
ssssgrp_name_fr                                légumes cruslégumes crus
name_fr                           avocat, pulpe, crucéleri branche, cru
grp_code                                                              4
ssgrp_code                                                          402
ssssgrp_code                                                      40202
code                                                              33027
energy_kcal                                                       222.6
water_g                                                           163.7
prot_g                                                             2.19
glucid_g                                                           3.24
lipid_g                                                            20.6
sugar_g                                                         